In [1]:
# -*- coding: utf-8 -*-
"""
Created on Sat Apr 18 20:18:15 2020

@author: jeffr
"""

# packages needed
import os
import pandas as pd

from pytube import YouTube
from moviepy.video.io.ffmpeg_tools import ffmpeg_extract_subclip
from moviepy.editor import *


In [19]:
# import dataframe of powerhour videos
df = pd.read_csv('./powerhourmaster.csv')

print(df.head(5))

# import list of selected clips for powerhour

ph = pd.read_csv('./powerhourlist.csv')

print(ph.head(5))


         sport  number        clipid    year          event  \
0  HorseRacing       1  HorseRacing1  2015.0  Triple crown    
1         Golf       1         Golf1  2019.0        Masters   
2         Golf       2         Golf2  2019.0        US Open   
3         Golf       5         Golf5  2018.0            ToC   
4         Golf       6         Golf6  2017.0        Masters   

                                         description  \
0                                   American Pharaoh   
1  Tiger wins 2019 masters (water balls by compet...   
2                       Woodland chip from the green   
3                          DJ near hole in one par 4   
4                            Sergio wins the masters   

                                          link  Start: Hours  Start: Mins  \
0  https://www.youtube.com/watch?v=WyDcXtVb8_U             0            1   
1  https://www.youtube.com/watch?v=pE-aOBGN8UM             0            0   
2  https://www.youtube.com/watch?v=LEXoR1MvnFo       

In [25]:
# remove video data not needed for power hour
notneeded = set(df.clipid) - set(ph.clipid)

print(set(df.clipid))
print((notneeded))

df = df[~df.clipid.isin(notneeded)]

print(set(df.clipid))

VIDEO_STAGE_PATH = './video_staging/'


{'Golf8', 'Golf11', 'Golf1', 'Golf5', 'Golf9', 'Golf2', 'HorseRacing1', 'Golf10', 'Golf6', 'Golf7'}
set()
{'Golf8', 'Golf11', 'Golf1', 'Golf5', 'Golf9', 'Golf2', 'HorseRacing1', 'Golf10', 'Golf6', 'Golf7'}


In [26]:
# initialize empty dataframe to put metadata in
video_length_logging = pd.DataFrame(columns=['video_title', 'video_start', 'video_finish'])

#  download youtube videos

for clip in range(0,len(df)):
        
    video_link = df.iloc[clip].link
    clipid = df.iloc[clip].clipid
    yt = YouTube(video_link)

    # grab only video (mp4) files
    mp4files = yt.streams.filter('mp4')
    #
    d_video = yt.streams.first()#(mp4files[-1].extension,mp4files[-1].resolution)
    
    # build some metadata for slicing videos later
    video_title = d_video.title
    video_start = df.iloc[clip].start
    video_finish = float(video_start) + df.iloc[clip].duration
    video_length_logging = video_length_logging.append(
        {'video_title': clipid, 
         'video_start': video_start, 
         'video_finish': video_finish}
        , ignore_index=True)
    
    try: 
        #downloading the video 
        d_video.download(VIDEO_STAGE_PATH, filename=clipid) 
    except: 
        print("Some Error!") 
    print('Task Completed!')


Task Completed!
Task Completed!
Task Completed!
Task Completed!
Task Completed!
Task Completed!
Task Completed!
Task Completed!
Task Completed!
Task Completed!


In [30]:
print(os.listdir('video_staging/'))
print(video_length_logging)

['Golf1.mp4', 'Golf10.mp4', 'Golf11.mp4', 'Golf2.mp4', 'Golf5.mp4', 'Golf6.mp4', 'Golf7.mp4', 'Golf8.mp4', 'Golf9.mp4', 'HorseRacing1.mp4']
    video_title video_start  video_finish
0  HorseRacing1         101         161.0
1         Golf1           0          70.0
2         Golf2         945         995.0
3         Golf5           0          39.0
4         Golf6       14445       14505.0
5         Golf7           0          80.0
6         Golf8           0          49.0
7         Golf9           0          68.0
8        Golf10         209         269.0
9        Golf11          18          75.0


In [31]:
# clip videos to relevant time periods

if not os.path.exists('video_clipping'):
    os.makedirs('video_clipping')

for video_clip_name in os.listdir('video_staging/'):
    
    video_metadata = video_length_logging[video_length_logging['video_title']==video_clip_name.split(".mp4",1)[0]]
    
    test_clip = "video_staging/" + video_clip_name
    trimmed_clip = ffmpeg_extract_subclip(test_clip,
                                          float(video_metadata['video_start']),
                                          float(video_metadata['video_finish']),
                                          targetname='./video_clipping/' + video_clip_name)


Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful


In [33]:
if not os.path.exists('outputs'):
    os.makedirs('outputs')

# Intialize empty set for videos
L =[]

for root, dirs, files in os.walk("video_clipping/"):

    files.sort(reverse=True)

    for file in files:
        if os.path.splitext(file)[1] == '.mp4':
            filePath = os.path.join(root, file)
            video = VideoFileClip(filePath)
            L.append(video)

final_clip = concatenate_videoclips(L)
final_clip.to_videofile("outputs/output.mp4", fps=24, remove_temp=False)

chunk:   0%|          | 19/13153 [00:00<01:12, 182.17it/s, now=None]

Moviepy - Building video outputs/output.mp4.
MoviePy - Writing audio in outputTEMP_MPY_wvf_snd.mp3



chunk:  18%|█▊        | 2334/13153 [00:12<00:54, 198.35it/s, now=None]


chunk:  33%|███▎      | 4348/13153 [00:25<00:59, 147.90it/s, now=None]


chunk:  48%|████▊     | 6341/13153 [00:37<00:35, 189.55it/s, now=None]


...

t:  78%|███████▊  | 11120/14316 [01:53<00:45, 70.65it/s, now=None]


t:  91%|█████████ | 13023/14316 [02:07<00:12, 100.96it/s, now=None]


t: 100%|█████████▉| 14255/14316 [02:21<00:00, 92.43it/s, now=None]


t:   0%|          | 2/14316 [04:44<565:17:55, 142.17s/it, now=None][A

Moviepy - Done !
Moviepy - video ready outputs/output.mp4
